In [ ]:
import warnings
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Ignore warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/Shareddrives/Hackathon/btc_4h.csv'
df = pd.read_csv("/content/drive/Shareddrives/Hackathon/btc_4h.csv")

In [ ]:
df

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13155.38,13410.03,1676.204807
1,2018-01-01 09:30:00,13434.98,13818.55,13322.15,13570.35,1302.214836
2,2018-01-01 13:30:00,13569.98,13735.24,13001.13,13220.56,1319.755931
3,2018-01-01 17:30:00,13220.56,13330.00,12750.00,13247.00,1831.933153
4,2018-01-01 21:30:00,13247.00,13290.65,12940.00,13240.37,1092.337234
...,...,...,...,...,...,...
8926,2022-01-30 13:30:00,38244.98,38359.26,37795.00,38110.02,4712.873740
8927,2022-01-30 17:30:00,38110.03,38153.82,37712.05,37980.09,2934.129800
8928,2022-01-30 21:30:00,37977.47,38074.76,37442.52,37512.37,3351.248670
8929,2022-01-31 01:30:00,37512.37,38230.00,37351.63,37881.76,4375.453430


In [ ]:
# Function to print metrics for regression or classification
def metrics(text, y, y_pred, reg=False):
    print(f"{text} Set Metrics:")

    # R2 Score is applicable for both regression and classification
    r2_score_ = r2_score(y, y_pred)
    print(f"R2 Score: {r2_score_}")

    if reg:
        # Regression metrics
        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        print(f"Mean Squared Error: {mse}")
        print(f"Mean Absolute Error: {mae}")
    else:
        # Classification metrics
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(y, y_pred, average='micro')
        recall = recall_score(y, y_pred, average='micro')
        f1_score_ = f1_score(y, y_pred, average='micro')

        # Print the classification metrics
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1_score_}")
    print()

In [ ]:
import numpy as np
import pandas as pd

# Assuming df is your DataFrame
X = df.iloc[:, 1].values
# y = list(df.iloc[:, -1])

# Function to create regression sequences
def create_regression_sequences(data, lookback, forecast_size):
    sequences, targets = [], []
    for i in range(len(data) - lookback - forecast_size):
        # Extract a sequence of 'lookback' days as input
        seq = data[i:i + lookback]

        # Extract the next 'forecast_size' days as the target/label
        label = data[i + lookback:i + lookback + forecast_size]

        # Append the sequence and target to the lists
        sequences.append(seq)
        targets.append(label)

    return np.array(sequences), np.array(targets)

import numpy as np



# Define the lookback window and forecast size
lookback_window = 10
forecast_size = 1

# Create regression sequences
X_reg, y_reg = create_regression_sequences(X, lookback_window, forecast_size)
# X_reg, y_reg = create_regression_sequences_with_ma(X, lookback_window, forecast_size)
# X_cls, y_cls = create_classification_sequences(X, lookback_window)

#Split into training and testing data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42, shuffle=False)
# X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
np.shape(X_train_reg) ,np.shape(X_test_reg)

((7136, 10), (1784, 10))

In [ ]:
# Initialize the RandomForestRegressor with the best parameters
regressor = RandomForestRegressor(n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        criterion="squared_error",
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features="auto",
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        bootstrap=True,
        oob_score=False,
        n_jobs=None,
        verbose=0,
        warm_start=False,
        ccp_alpha=0.0,
        max_samples=None,
        random_state=12)

# Train the regressor on the training set
regressor.fit(X_train_reg, y_train_reg)

# Make predictions on the test set
y_pred_train_rf = regressor.predict(X_train_reg)
y_pred_test_rf = regressor.predict(X_test_reg)

# Evaluate the model on the test set
metrics('Test', y_pred_test_rf, y_test_reg, True)
metrics('Train', y_pred_train_rf, y_train_reg, True)

Train Set Metrics:
R2 Score: 0.9998820645527133
Mean Squared Error: 14760.061048243917
Mean Absolute Error: 54.92394429652503

Test Set Metrics:
R2 Score: 0.9693747358366076
Mean Squared Error: 2456163.9950162075
Mean Absolute Error: 1038.8512681053812



In [ ]:
import numpy as np
import plotly.graph_objects as go

# Reshape arrays for plotting
y_test_reg_array = np.array(y_test_reg).reshape(-1)
y_train_reg_array = np.array(y_train_reg).reshape(-1)
y_pred_test_rf_array = np.array(y_pred_test_rf).reshape(-1)
y_pred_train_rf_array = np.array(y_pred_train_rf).reshape(-1)

# Plotting the results using Plotly
fig = go.Figure()

# Add true values and predicted values traces for the test set
fig.add_trace(go.Scatter(x=list(range(len(y_train_reg), len(y_test_reg) + len(y_train_reg))),
                         y=y_test_reg_array,
                         mode='lines',
                         name='True Values (Test)'))

fig.add_trace(go.Scatter(x=list(range(len(y_train_reg), len(y_test_reg) + len(y_train_reg))),
                         y=y_pred_test_rf_array,
                         mode='lines',
                         name='Predicted Values (Test)'))

# Add true values and predicted values traces for the train set
fig.add_trace(go.Scatter(x=list(range(len(y_train_reg))),
                         y=y_train_reg_array,
                         mode='lines',
                         name='True Values (Train)'))

fig.add_trace(go.Scatter(x=list(range(len(y_train_reg))),
                         y=y_pred_train_rf_array,
                         mode='lines',
                         name='Predicted Values (Train)'))

# Update layout
fig.update_layout(title='Random Forest Regressor Prediction',
                  xaxis_title='Time',
                  yaxis_title='Values')

# Show the plot
fig.show()


In [ ]:
# Function to implement a simple gap trading strategy
def gap_trading_strategy(prices, gap_threshold_percent=1.15):
    # Create a DataFrame with 'Close' prices
    data = pd.DataFrame(prices, columns=['Close'])

    # Calculate price change and identify gaps
    data['PriceChange'] = data['Close'].diff()
    data['Gap'] = np.where((data['PriceChange'].shift(-1) / data['Close']) * 100 > gap_threshold_percent, 1.0, 0.0)

    # Determine position based on gap
    data['Position'] = data['Gap'].diff()

    return data
def merge_and_calculate_profit(X_test_reg, regressor, initial_balance=100000,gap_threshold_percent=1.15,stop_loss_percent=0.013):
    # Initialize variables and parameters
    history = []  # List to store historical prices and forecasts
    history.extend(X_test_reg[0])  # Initialize history with the first observation
    history.pop()  # Remove the last element to maintain the desired lookback window
    actions = []  # List to store trading actions
    position = 0  # Current trading position (0: neutral, 1: long, -1: short)
    total_profit = 0  # Total profit accumulated during trading
    balance = initial_balance  # Initial balance for trading
    stop_loss = 700  # Initial stop-loss value
    shares_bought = 0 # Initial number of shares bought
    shares_sold_short = 0
    buy_price =0
    price = 0
    # Create an empty DataFrame to store the log
    log_df = pd.DataFrame(columns=['open', 'signal'])

    # Loop through each observation in the test set
    for idx, i in enumerate(X_test_reg):
        price = i[-1]
        history.append(price)
        forecast = regressor.predict([i])[0]
        history.append(forecast)

        # Apply the gap trading strategy to get the current trading action
        gap_strategy_data = gap_trading_strategy(history, gap_threshold_percent)
        history.pop()

        current_action = gap_strategy_data.iloc[-2, -1]

        # Execute trading actions based on the current position and trading strategy
        if position == 1:
            if buy_price - price > stop_loss:
                # Sell
                position = 0
                profit_loss = shares_bought * (price - buy_price)
                total_profit += profit_loss
                balance = shares_bought * price
                shares_bought = 0
                log_df = log_df.append({'open': price,'signal': -1,'Stop_loss':stop_loss,  'Model_Action':'Stop Loss', 'Forecast': forecast, 'Action': 'Sell', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': profit_loss},
                                       ignore_index=True)
            elif current_action == 1:
                # Hold
                log_df = log_df.append(
                    {'open': price,'signal': 0,'Stop_loss':stop_loss,  'Model_Action':'Buy',  'Forecast': forecast , 'Action': 'Hold', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                    ignore_index=True)
            elif current_action == -1:
                # Sell
                position = 0
                profit_loss = shares_bought * (price - buy_price)
                total_profit += profit_loss
                balance = shares_bought * price
                shares_bought = 0
                log_df = log_df.append({'open': price,'signal': -1,'Stop_loss':stop_loss,  'Model_Action':'Sell', 'Forecast': forecast, 'Action': 'Sell', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': profit_loss},
                                       ignore_index=True)
                # Short Sell
                position = -1
                share_price_short = price
                shares_sold_short = balance / price
                short_sell_price = price * shares_sold_short
                shares_bought = -shares_sold_short
                balance = 0
                stop_loss = share_price_short*stop_loss_percent
                log_df = log_df.append({'open': price,'signal': -1,'Stop_loss':stop_loss,  'Model_Action':'Sell Short Sell', 'Forecast': forecast, 'Action': 'Sell', 'Current Position':shares_bought , 'Balance': balance, 'Profit/Loss': 0},
                                       ignore_index=True)
            else:
                # Hold
                log_df = log_df.append(
                    {'open': price,'signal': 0,'Stop_loss':stop_loss,  'Model_Action':'Hold', 'Forecast': forecast, 'Action': 'Hold', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                    ignore_index=True)

        elif position == 0:
            if current_action == 1:
                # Buy
                position = 1
                buy_price = price
                shares_bought = balance / buy_price
                balance = 0
                stop_loss = buy_price * stop_loss_percent
                log_df = log_df.append({'open': price,'signal': 1,'Stop_loss':stop_loss,  'Model_Action':'Buy', 'Forecast': forecast, 'Action': 'Buy', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                                       ignore_index=True)
            elif current_action == -1:

                # Hold
                log_df = log_df.append(
                    {'open': price,'signal': 0,'Stop_loss':stop_loss,  'Model_Action':'Sell', 'Forecast': forecast, 'Action': 'Hold', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                    ignore_index=True)
            else:
                # Hold
                log_df = log_df.append(
                    {'open': price,'signal': 0,'Stop_loss':stop_loss,  'Model_Action':'Hold', 'Forecast': forecast, 'Action': 'Hold', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                    ignore_index=True)
        else:
            # print(idx,price - share_price_short , stop_loss)
            if price - share_price_short > stop_loss:
                # Close short sell
                position = 0
                profit_loss = shares_sold_short * (share_price_short - price)
                total_profit += profit_loss
                balance = short_sell_price + profit_loss
                shares_bought=0
                log_df = log_df.append({'open': price, 'signal': 1,'Stop_loss':stop_loss,  'Model_Action':'Buy Close short sell', 'Forecast': forecast, 'Action': 'Buy', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': profit_loss},
                                       ignore_index=True)
                # Buy
                position = 1
                buy_price = price
                shares_bought = balance / buy_price
                balance = 0
                stop_loss = buy_price * stop_loss_percent
                log_df = log_df.append({'open': price,'signal': 1,'Stop_loss':stop_loss,  'Model_Action':'Buy', 'Forecast': forecast, 'Action': 'Buy', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                                       ignore_index=True)

            elif current_action == 1:
                # Close short sell
                position = 0
                profit_loss = shares_sold_short * (share_price_short - price)
                total_profit += profit_loss
                balance = short_sell_price + profit_loss
                shares_bought = 0
                log_df = log_df.append({'open': price, 'signal': 1,'Stop_loss':stop_loss,  'Model_Action':'Close short sell stop loss', 'Forecast': forecast, 'Action': 'Buy', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': profit_loss},
                                       ignore_index=True)
                # Buy
                position = 1
                buy_price = price
                shares_bought = balance / buy_price
                balance = 0
                stop_loss = buy_price * stop_loss_percent
                log_df = log_df.append({'open': price,'signal': 1,'Stop_loss':stop_loss,  'Model_Action':'Buy', 'Forecast': forecast, 'Action': 'Buy', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                                       ignore_index=True)

            elif current_action == -1:
                # Hold
                log_df = log_df.append(
                    {'open': price,'signal': 0,'Stop_loss':stop_loss,  'Model_Action':'Sell', 'Forecast': forecast, 'Action': 'Hold', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                    ignore_index=True)
            else:
                # Hold
                log_df = log_df.append(
                    {'open': price,'signal': 0,'Stop_loss':stop_loss,  'Model_Action':'Hold', 'Forecast': forecast, 'Action': 'Hold', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': 0},
                    ignore_index=True)
    # If there's an open position at the end, close it at the last price
    if position == 1:
        # Sell
        profit_loss = shares_bought * (price - buy_price)
        total_profit += profit_loss
        log_df.drop(log_df.tail(1).index, inplace=True)
        log_df = log_df.append(
            {'open': price,'signal': -1,'Model_Action': 'Sell', 'Action': 'Sell', 'Current Position': 0,
             'Balance': shares_bought * price, 'Profit/Loss': profit_loss}, ignore_index=True)

    elif position == -1:
        # Close short sell
        position = 0
        profit_loss = shares_sold_short * (share_price_short - price)
        total_profit += profit_loss
        balance = short_sell_price + profit_loss
        shares_bought=0+6
        log_df.drop(log_df.tail(1).index, inplace=True)
        log_df = log_df.append({'open': price,'signal': 1,'Stop_loss':stop_loss,  'Model_Action':'Close short sell', 'Action': 'Buy', 'Current Position': shares_bought, 'Balance': balance, 'Profit/Loss': profit_loss},
                                       ignore_index=True)
    return log_df, total_profit, history,gap_strategy_data


In [ ]:
# Use the trained regression model to merge and calculate profits
log_df, total_profit, history,gap_strategy_data = merge_and_calculate_profit(X_test_reg, regressor,gap_threshold_percent=0.85,stop_loss_percent=0.030)

# Print the total profit obtained during the trading period
print("Total Profit:", total_profit)

Total Profit: 483996.35887765244


In [ ]:
new_df = df.iloc[len(X_test_reg)*(-1)-2:]
df2= log_df.iloc[:,:2]
merged_df = pd.merge(new_df, df2, on='open', how='left')
merged_df['signal']=merged_df['signal'].fillna(0)

In [ ]:
merged_df[merged_df.isna().any(axis=1)]

,datetime,open,high,low,close,volume,signal


In [ ]:
merged_df

,datetime,open,high,low,close,volume,signal
0,2021-04-08 17:30:00,56421.17,57912.02,56311.77,57768.13,10949.664726,0
1,2021-04-08 21:30:00,57769.00,58153.31,57538.52,57734.94,6729.021841,0
2,2021-04-09 01:30:00,57732.53,58100.00,57449.04,58077.52,4832.703939,0
3,2021-04-09 05:30:00,58077.52,58419.45,57806.03,57985.71,5235.243334,0
4,2021-04-09 09:30:00,57985.71,58314.00,57654.00,58051.42,5646.760536,0
...,...,...,...,...,...,...,...
2012,2022-01-30 13:30:00,38244.98,38359.26,37795.00,38110.02,4712.873740,0
2013,2022-01-30 17:30:00,38110.03,38153.82,37712.05,37980.09,2934.129800,0
2014,2022-01-30 21:30:00,37977.47,38074.76,37442.52,37512.37,3351.248670,1
2015,2022-01-31 01:30:00,37512.37,38230.00,37351.63,37881.76,4375.453430,0


In [ ]:
merged_df.to_csv('Amigos_IIT_BHU.csv')

In [ ]:
def map_actions_to_colors(log_df):
    # Map actions to colors
    log_df_copy = log_df.copy()
    # log_df_copy.loc[(log_df_copy['Action'] == 'Sell') & (log_df_copy['Profit/Loss'] < 0), 'Action'] = 'Lost_sell'
    action_colors = {'Buy': 'green', 'Sell': 'red', 'Hold': 'Blue'}
    log_df_copy['Color'] = log_df_copy['Action'].map(action_colors)
    return log_df_copy

def plot_open_vs_action(log_df):
    # Create line plot for opens
    fig = go.Figure()

    # Plot opens as lines
    fig.add_trace(go.Scatter(x=log_df.index, y=log_df['open'], mode='lines', name='open'))

    # Plot Buy and Sell actions as markers
    action_colors = {'Buy': 'green', 'Sell': 'red'}
    for action, color in action_colors.items():
        if action in log_df['Action'].unique():
            subset = log_df[log_df['Action'] == action]
            fig.add_trace(go.Scatter(x=subset.index, y=subset['open'],
                                     mode='markers', marker=dict(color=color),
                                     name=action))

    # Customize the layout
    fig.update_layout(title='open vs. Action (Buy and Sell)',
                      xaxis_title='Index',
                      yaxis_title='open_price',
                      showlegend=True)

    # Show the plot
    fig.show()
log_df_mapped = map_actions_to_colors(log_df)
plot_open_vs_action(log_df_mapped)


In [ ]:
df = log_df.copy()

In [ ]:
df['signal'].value_counts()

 0    1479
 1     268
-1     268
Name: signal, dtype: int64

In [ ]:
df[df['Model_Action']=='Sell']['Profit/Loss'].sum()

756319.1738641863

In [ ]:
import pandas as pd
import numpy as np

# 1. Gross Profit Calculation
gross_profit = df[df['Profit/Loss'] > 0]['Profit/Loss'].sum()

# 2. Net Profit Calculation
net_profit = df['Profit/Loss'].sum()

# 3. Gross Loss Calculation
gross_loss = df[df['Profit/Loss'] < 0]['Profit/Loss'].sum()

# 4. Max Drawdown Calculation
# Convert 'Profit/Loss' column to numeric type
df['Profit/Loss'] = pd.to_numeric(df['Profit/Loss'], errors='coerce')
# Find the index of the minimum value
min_index = df['Profit/Loss'].idxmin()
# Get the maximum drawdown value and corresponding balance
max_drawdown = df['Profit/Loss'][min_index]

# 5. Buy and Hold Return of BTC Calculation
balance_at_start = 100000
balance_at_end = df['Balance'].iloc[-1]
buy_and_hold_return = ((balance_at_end - balance_at_start) / balance_at_start) * 100

# 6. Sharpe Ratio Calculation
returns = (df[df['Balance'] > 0]['Balance'] - balance_at_start) / balance_at_start
risk_free_rate = 0.1
portfolio_excess_return = returns - risk_free_rate
sharpe_ratio = np.mean(portfolio_excess_return) / np.std(portfolio_excess_return)

# 7. Sortino Ratio Calculation
downside_returns = np.minimum(returns - risk_free_rate, 0)
downside_deviation = np.std(downside_returns)
sortino_ratio = (np.mean(returns) - risk_free_rate) / downside_deviation if downside_deviation != 0 else np.nan

# 8. Total Closed Trades Calculation
total_closed_trades = df[df['Profit/Loss'] != 0].shape[0]

# 9. Number of Winning Trades Calculation
profit_trades = df[df['Profit/Loss'] > 0].shape[0]

# 10. Number of Losing Trades Calculation
loss_trades = df[df['Profit/Loss'] < 0].shape[0]

# 11. Average Winning Trade Calculation
average_winning_trade = df[df['Profit/Loss'] > 0]['Profit/Loss'].mean()

# 12. Average Losing Trade Calculation
average_losing_trade = df[df['Profit/Loss'] < 0]['Profit/Loss'].mean()

# 13. Largest Winning Trade Calculation
largest_winning_trade = df[df['Profit/Loss'] > 0]['Profit/Loss'].max()

# 14. Largest Losing Trade Calculation
largest_losing_trade = df[df['Profit/Loss'] < 0]['Profit/Loss'].min()

# 15. Average Holding Duration per Trade Calculation (in 4hrs)
df['Trade_Duration'] = 0
# Initialize variables
in_trade = False
trade_start_index = 0
trade_durations = []
# Iterate through the DataFrame
for i in range(1, len(df)):
    if df['Current Position'][i] > 0 and df['Action'][i] == 'Buy' and not in_trade:
        # Start counting duration
        in_trade = True
        trade_start_index = i
    elif df['Current Position'][i] < 0 and df['Action'][i] == 'Sell' and not in_trade:
        # Start counting duration for short sell
        in_trade = True
        trade_start_index = i
    elif in_trade and ((df['Action'][i] == 'Sell' and df['Current Position'][i] == 0) or
                       (df['Action'][i] == 'Buy' and df['Current Position'][i] == 0)):
        # Stop counting duration and append to the list
        in_trade = False
        trade_duration = i - trade_start_index
        trade_durations.append(trade_duration)
average_trade_time = sum(trade_durations) / len(trade_durations) if trade_durations else 0

# 16. Max Dip and Average Dip Calculation
Max_DIP = balance_at_start - df[df['Balance'] > 0][['Balance']].min()
Average_dip = balance_at_start - df[df['Balance'] > 0][df['Balance'] < balance_at_start][['Balance']].mean()

# Print the results with proper formatting
print(f"1. Gross Profit: {gross_profit:.2f}")
print(f"2. Net Profit: {net_profit:.2f}")
print(f"3. Gross Loss: {gross_loss:.2f}")
print(f"4. Max Drawdown: {max_drawdown:.2f}")
print(f"5. Buy and Hold Return of BTC: {buy_and_hold_return:.2f}%")
print(f"6. Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"7. Sortino Ratio: {sortino_ratio:.4f}")
print(f"8. Total Closed Trades: {total_closed_trades}")
print(f"9. Number of Winning Trades: {profit_trades}")
print(f"10. Number of Losing Trades: {loss_trades}")
print(f"11. Average Winning Trade (in USDT): {average_winning_trade:.2f}")
print(f"12. Average Losing Trade (in USDT): {average_losing_trade:.2f}")
print(f"13. Largest Winning Trade (in USDT): {largest_winning_trade:.2f}")
print(f"14. Largest Losing Trade (in USDT): {largest_losing_trade:.2f}")
print(f"15. Average Holding Duration per Trade: {average_trade_time*4:.2f} hrs")  # Assuming trade duration is in days
print(f"16. Max Dip and Average Dip: {np.array(Max_DIP).astype(float)[0]:.2f}, {np.array(Average_dip).astype(float)[0]:.2f}")


1. Gross Profit: 1484430.98
2. Net Profit: 483996.36
3. Gross Loss: -1000434.62
4. Max Drawdown: -35734.68
5. Buy and Hold Return of BTC: 484.00%
6. Sharpe Ratio: 1.2384
7. Sortino Ratio: 112.3862
8. Total Closed Trades: 268
9. Number of Winning Trades: 171
10. Number of Losing Trades: 97
11. Average Winning Trade (in USDT): 8680.88
12. Average Losing Trade (in USDT): -10313.76
13. Largest Winning Trade (in USDT): 54753.93
14. Largest Losing Trade (in USDT): -35734.68
15. Average Holding Duration per Trade: 23.97 hrs
16. Max Dip and Average Dip: 1263.34, 1263.34


In [ ]:
min_index = df['Profit/Loss'].idxmin()
# Get the maximum drawdown value and corresponding balance
max_drawdown = df['Profit/Loss'][min_index]
# Get the value at max_drawdown
Balance_at_max_drawdown = df['Balance'][min_index]
# Print the max drawdown value and percentage
print(f"Max Drawdown Value: {max_drawdown:.2f}")
print(f"Max Drawdown Percentage: {max_drawdown / Balance_at_max_drawdown * 100:.2f}%")


Max Drawdown Value: -35734.68
Max Drawdown Percentage: -6.28%


In [ ]:
Balance_at_max_drawdown,min_index

(568716.2842860637, 1685)

In [ ]:
# Assuming start_price and end_price are the initial and final prices
start_price = X_test_reg[0][-1]
end_price = X_test_reg[-1][-1]

# Initial cash and number of stocks purchased
initial_cash = 100000
number_of_stocks_purchased = initial_cash / start_price

# Calculate the Benchmark Return
benchmark_return = (number_of_stocks_purchased * end_price - initial_cash) / initial_cash

# Print the Benchmark Return
print(f"Benchmark Return: {benchmark_return:.2f}")

# Calculate your return
balance_at_start = 100000
balance_at_end = log_df['Balance'].iloc[-1]
your_return = ((balance_at_end - balance_at_start) / balance_at_start)

# Calculate the Percentage Increase
percentage_increase = ((your_return - benchmark_return) / abs(benchmark_return)) * 100

# Print the Percentage Increase
print(f"Your Return: {your_return:.2f}")
print(f"Percentage Increase from Benchmark: {percentage_increase:.2f}%")


Benchmark Return: -0.33
Your Return: 4.84
Percentage Increase from Benchmark: 1580.59%


In [ ]:
print(f'Risk-Reward Ratio of the Model : {profit_trades/loss_trades}')

Risk-Reward Ratio of the Model : 1.7628865979381443


In [ ]:
print(f'Max Duration Time of Single Trade of the Model {max(trade_durations)*4/24} days')

Max Duration Time of Single Trade of the Model 8.166666666666666 days
